<a href="https://colab.research.google.com/github/stefsyrsiri/Deep-Learning-Project/blob/import-ipynb/Deep_Learning_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
print("This is just the beginning.")

This is just the beginning.
